# bet365-scraper

In this script, we scrap football information from bet365

In [65]:
### imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [82]:
### connect to bet365.com and travel to games list
# connect to site
url = 'https://www.bet365.com/en/'
driver = webdriver.Firefox()
driver.get(url)

# click on sports banner, once it is loaded
sports_banner = WebDriverWait(driver,5).until(
    EC.presence_of_element_located((By.ID, 'dv1'))
)
sports_banner.click()

# click on 'Elite Euro List', once previous page is loaded
top_coupons = WebDriverWait(driver,5).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'tc-TopCouponStemRenderer'))
)
top_coupons.find_elements_by_class_name('tc-TopCouponLinkButton')[1].click()

# change odds to decimal
odds_selector = WebDriverWait(driver,5).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'hm-OddsDropDownSelections.hm-DropDownSelections'))
)
odds_selector.click()

odds_items = WebDriverWait(driver,5).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'hm-DropDownSelections_Item'))
)
odds_items[1].click()

3